In [19]:
import openai
import config
import os 
import json
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import JSONLoader, DirectoryLoader, UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [3]:
key = "sk-jaadXZEd5xxxxxxxxxxxxxxxxxxx7Y5cAR58M"

In [4]:
os.environ["OPENAI_API_KEY"] = key

In [45]:
html_dir_loader = DirectoryLoader('/scratch/ac.pgupta/convLLM/kbasedocs', glob="**/[!.]*.html", loader_cls=UnstructuredHTMLLoader)
data_html = html_dir_loader.load()
len(data_html)

92

### document text example

In [48]:
print("""'KBase Documentation\n\nHere you will find guidance for getting started with KBase, full documentation for users and App developers, and help with troubleshooting.\n\nHow to cite KBase:\n\nArkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al. KBase: The United States Department of Energy Systems Biology Knowledgebase. Nature Biotechnology. 2018;36: 566. doi: 10.1038/nbt.4163\n\nFunding:\n\nThis work is supported as part of the Genomic Sciences Program DOE Systems Biology Knowledgebase (KBase) funded by the\n\nU.S. Department of Energy, Office of Science, Office of Biological and Environmental Research\n\nunder Award Numbers DE-AC02-05CH11231, DE-AC02-06CH11357, DE-AC05-00OR22725, and DE-AC02-98CH10886.\n\nNextKBase Terms & Conditions\n\nLast modified \n\n2yr ago', metadata={'source': '/scratch/ac.pgupta/convLLM/kbasedocs/index.html'""")

'KBase Documentation

Here you will find guidance for getting started with KBase, full documentation for users and App developers, and help with troubleshooting.

How to cite KBase:

Arkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al. KBase: The United States Department of Energy Systems Biology Knowledgebase. Nature Biotechnology. 2018;36: 566. doi: 10.1038/nbt.4163

Funding:

This work is supported as part of the Genomic Sciences Program DOE Systems Biology Knowledgebase (KBase) funded by the

U.S. Department of Energy, Office of Science, Office of Biological and Environmental Research

under Award Numbers DE-AC02-05CH11231, DE-AC02-06CH11357, DE-AC05-00OR22725, and DE-AC02-98CH10886.

NextKBase Terms & Conditions

Last modified 

2yr ago', metadata={'source': '/scratch/ac.pgupta/convLLM/kbasedocs/index.html'


In [49]:
print("""Supported Browsers\n\nThe KBase website and Narrative Interface should work on most modern web browsers, but we primarily focus on supporting Chrome.\n\nSupported Browsers\n\nThe following desktop web browsers have been tested against the KBase web site and Narrative user interface. If you experience any issue using these browsers with our site please report it through the\n\nHelp Board\n\nOS Supported Browsers Mac Chrome Firefox Safari 10 (macOS Sierra) and up Windows Firefox Microsoft Edge - not recommended Linux Chrome Firefox\n\nAd blockers may prevent KBase from working in your web browser\n\nSome ad-blockers and browser extensions may cause errors within the Narrative Interface. We are working to identify issues with commonly used browser extensions, but you may want to disable browser extensions if you encounter issues within KBase.\n\nMobile\n\nKBase is currently not supported on mobile browsers. The KBase website and services should work with tablet browsers on iPad and Android devices in landscape orientation, but we do not currently test on these platforms so there may be compatibility issues. Watch for future updates as KBase goes responsive and mobile.\n\nInstallation of Supported Browsers\n\nChrome\n\nChrome is the best browser to use for KBase. The Chrome browser is produced by Google, Inc., and is freely available. Chrome operates on a rolling release, meaning that it is updated frequently and it is not easy to track specific versions. Our best advice is to make sure you are using the most recent version available. On Mac and Windows this is built into the Chrome application itself.\n\n\u200bDownload Chrome\u200b\n\nFirefox\n\nFirefox, produced by the non-profit Mozilla Corporation, is a freely available, open source browser for Mac, Windows, Linux, and other systems. Like Chrome, Firefox is updated frequently, so it is not feasible for us to recommend a specific (or even minimum) version. Our best advice is to make sure you are using the most recent version available. Recent versions of Firefox will automatically update themselves on Mac and Windows.\n\n\u200bDownload Firefox\u200b\n\nSafari\n\nSafari is distributed with Mac OS X. (We do not support the Windows version of Safari.) Specific versions of Mac OS X support specific versions of Safari – see this\n\nWikipedia entry\n\nfor details.\n\nMicrosoft Edge\n\nMicrosoft Edge\n\nis the new web browser for Windows 10; it replaces Internet Explorer. You should be able to use KBase on Edge, but this has not been extensively tested.\n\nUnsupported Browsers\n\nKBase is not supported on Internet Explorer. It should work on Microsoft Edge, but it has not been extensively tested. For the best chance of success on Edge, be sure you are using the most recent version.\n\nOlder versions of supported browsers are also not supported. Please ensure that your browser and system are up-to-date before reporting a problem with a supported browser.\n\nOther modern browsers, such as Opera, may work with KBase, but as we do not test with them we cannot make any assurances about their operation.\n\nPreviousAuthentication Update\n\nNextNarrative Quick Start\n\nLast modified \n\n1yr ago""")

Supported Browsers

The KBase website and Narrative Interface should work on most modern web browsers, but we primarily focus on supporting Chrome.

Supported Browsers

The following desktop web browsers have been tested against the KBase web site and Narrative user interface. If you experience any issue using these browsers with our site please report it through the

Help Board

OS Supported Browsers Mac Chrome Firefox Safari 10 (macOS Sierra) and up Windows Firefox Microsoft Edge - not recommended Linux Chrome Firefox

Ad blockers may prevent KBase from working in your web browser

Some ad-blockers and browser extensions may cause errors within the Narrative Interface. We are working to identify issues with commonly used browser extensions, but you may want to disable browser extensions if you encounter issues within KBase.

Mobile

KBase is currently not supported on mobile browsers. The KBase website and services should work with tablet browsers on iPad and Android devices in lands

In [6]:
llm = ChatOpenAI(temperature=0, model="gpt-4")

In [66]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
)
documents_html = text_splitter.split_documents(data_html)
len(documents_html)

408

In [67]:
documents_html[0]

Document(page_content='KBase Documentation\n\nHere you will find guidance for getting started with KBase, full documentation for users and App developers, and help with troubleshooting.\n\nHow to cite KBase:\n\nArkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al. KBase: The United States Department of Energy Systems Biology Knowledgebase. Nature Biotechnology. 2018;36: 566. doi: 10.1038/nbt.4163\n\nFunding:\n\nThis work is supported as part of the Genomic Sciences Program DOE Systems Biology Knowledgebase (KBase) funded by the\n\nU.S. Department of Energy, Office of Science, Office of Biological and Environmental Research\n\nunder Award Numbers DE-AC02-05CH11231, DE-AC02-06CH11357, DE-AC05-00OR22725, and DE-AC02-98CH10886.\n\nNextKBase Terms & Conditions\n\nLast modified \n\n2yr ago', metadata={'source': '/scratch/ac.pgupta/convLLM/kbasedocs/index.html'})

### Few shot example prompt

In [52]:
few_shot_examples = [
  {
    "Question": """Given the text provided, generate several questions and answers pairs from the text. 
                text is:
                KBase Documentation\n\nHere you will find guidance for getting started with KBase, full documentation for users and App developers, and help with troubleshooting.\n\nHow to cite KBase:\n\nArkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al. KBase: The United States Department of Energy Systems Biology Knowledgebase. Nature Biotechnology. 2018;36: 566. doi: 10.1038/nbt.4163\n\nFunding:\n\nThis work is supported as part of the Genomic Sciences Program DOE Systems Biology Knowledgebase (KBase) funded by the\n\nU.S. Department of Energy, Office of Science, Office of Biological and Environmental Research\n\nunder Award Numbers DE-AC02-05CH11231, DE-AC02-06CH11357, DE-AC05-00OR22725, and DE-AC02-98CH10886.\n\nNextKBase Terms & Conditions\n\nLast modified \n\n2yr ago
                """,

    "Answer":
    """
    1. ### question: How to cite KBase? ### answer: Arkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al. ;
    2. ### question: What is KBase? ### answer: The United States Department of Energy Systems Biology Knowledgebase.;
    3. ### question: Who supports and funds KBase? ### answer:The United States Department of Energy (DOE);

    """
  },
  {
    "Question": """Given the text provided, generate several questions and answers pairs from the text. 
                text is:
                Supported Browsers\n\nThe KBase website and Narrative Interface should work on most modern web browsers, but we primarily focus on supporting Chrome.\n\nSupported Browsers\n\nThe following desktop web browsers have been tested against the KBase web site and Narrative user interface. If you experience any issue using these browsers with our site please report it through the\n\nHelp Board\n\nOS Supported Browsers Mac Chrome Firefox Safari 10 (macOS Sierra) and up Windows Firefox Microsoft Edge - not recommended Linux Chrome Firefox\n\nAd blockers may prevent KBase from working in your web browser\n\nSome ad-blockers and browser extensions may cause errors within the Narrative Interface. We are working to identify issues with commonly used browser extensions, but you may want to disable browser extensions if you encounter issues within KBase.\n\nMobile\n\nKBase is currently not supported on mobile browsers. The KBase website and services should work with tablet browsers on iPad and Android devices in landscape orientation, but we do not currently test on these platforms so there may be compatibility issues. Watch for future updates as KBase goes responsive and mobile.\n\nInstallation of Supported Browsers\n\nChrome\n\nChrome is the best browser to use for KBase. The Chrome browser is produced by Google, Inc., and is freely available. Chrome operates on a rolling release, meaning that it is updated frequently and it is not easy to track specific versions. Our best advice is to make sure you are using the most recent version available. On Mac and Windows this is built into the Chrome application itself.\n\n\u200bDownload Chrome\u200b\n\nFirefox\n\nFirefox, produced by the non-profit Mozilla Corporation, is a freely available, open source browser for Mac, Windows, Linux, and other systems. Like Chrome, Firefox is updated frequently, so it is not feasible for us to recommend a specific (or even minimum) version. Our best advice is to make sure you are using the most recent version available. Recent versions of Firefox will automatically update themselves on Mac and Windows.\n\n\u200bDownload Firefox\u200b\n\nSafari\n\nSafari is distributed with Mac OS X. (We do not support the Windows version of Safari.) Specific versions of Mac OS X support specific versions of Safari – see this\n\nWikipedia entry\n\nfor details.\n\nMicrosoft Edge\n\nMicrosoft Edge\n\nis the new web browser for Windows 10; it replaces Internet Explorer. You should be able to use KBase on Edge, but this has not been extensively tested.\n\nUnsupported Browsers\n\nKBase is not supported on Internet Explorer. It should work on Microsoft Edge, but it has not been extensively tested. For the best chance of success on Edge, be sure you are using the most recent version.\n\nOlder versions of supported browsers are also not supported. Please ensure that your browser and system are up-to-date before reporting a problem with a supported browser.\n\nOther modern browsers, such as Opera, may work with KBase, but as we do not test with them we cannot make any assurances about their operation.\n\nPreviousAuthentication Update\n\nNextNarrative Quick Start\n\nLast modified \n\n1yr ago
                """,

    "Answer":
    """
    1. ### question: What browswers are supported by KBase? ### answer: 
    There are four browswers that are supported by KBase, they are
    1. Chrome:Chrome is the best browser to use for KBase. 
    2. Firefox: Firefox, produced by the non-profit Mozilla Corporation, is a freely available, open source browser for Mac, Windows, Linux, and other systems. 
    3. Safari: Safari is distributed with Mac OS X. (We do not support the Windows version of Safari.)
    4. Microsoft Edge: Microsoft Edgeis the new web browser for Windows 10; it replaces Internet Explorer. You should be able to use KBase on Edge, but this has not been extensively tested.
    2. ### question: What browswers are supported for Mac? ### answer: Chrome, Firefox and Safari 10 (macOS Sierra) and up
    3. ### question: Why Kbase is not working in my browswer? ### answer:Ad blockers may prevent KBase from working in your web browserSome ad-blockers and browser extensions may cause errors within the Narrative Interface. We are working to identify issues with commonly used browser extensions, but you may want to disable browser extensions if you encounter issues within KBase.

    """
  }
]

In [58]:
example_prompt = PromptTemplate(input_variables=["Question", "Answer"], template="Question: {Question}\nAnswer: {Answer}")

print(example_prompt.format(**few_shot_examples[0]))

Question: Given the text provided, generate several questions and answers pairs from the text. 
                text is:
                KBase Documentation

Here you will find guidance for getting started with KBase, full documentation for users and App developers, and help with troubleshooting.

How to cite KBase:

Arkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al. KBase: The United States Department of Energy Systems Biology Knowledgebase. Nature Biotechnology. 2018;36: 566. doi: 10.1038/nbt.4163

Funding:

This work is supported as part of the Genomic Sciences Program DOE Systems Biology Knowledgebase (KBase) funded by the

U.S. Department of Energy, Office of Science, Office of Biological and Environmental Research

under Award Numbers DE-AC02-05CH11231, DE-AC02-06CH11357, DE-AC05-00OR22725, and DE-AC02-98CH10886.

NextKBase Terms & Conditions

Last modified 

2yr ago
                
Answer: 
    1. ### question: How to cite KBase? ### answer: Arkin AP, Co

In [59]:
instruction_prompt="Given the text provided, generate several questions and answers pairs from the text. text is:"
prompt = FewShotPromptTemplate(
    examples=few_shot_examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)
prompt
print(prompt.format(input=instruction_prompt))

Question: Given the text provided, generate several questions and answers pairs from the text. 
                text is:
                KBase Documentation

Here you will find guidance for getting started with KBase, full documentation for users and App developers, and help with troubleshooting.

How to cite KBase:

Arkin AP, Cottingham RW, Henry CS, Harris NL, Stevens RL, Maslov S, et al. KBase: The United States Department of Energy Systems Biology Knowledgebase. Nature Biotechnology. 2018;36: 566. doi: 10.1038/nbt.4163

Funding:

This work is supported as part of the Genomic Sciences Program DOE Systems Biology Knowledgebase (KBase) funded by the

U.S. Department of Energy, Office of Science, Office of Biological and Environmental Research

under Award Numbers DE-AC02-05CH11231, DE-AC02-06CH11357, DE-AC05-00OR22725, and DE-AC02-98CH10886.

NextKBase Terms & Conditions

Last modified 

2yr ago
                
Answer: 
    1. ### question: How to cite KBase? ### answer: Arkin AP, Co

In [61]:
system_message = SystemMessage(content=prompt.format(input=instruction_prompt))

text = """
Job Browser\n\nThe Job Browser is one of the buttons on the left sidebar menu in KBase. In addition to watching jobs within the Narratives, the Job Browser allows you to monitor and manage your jobs. By default, it shows all jobs submitted within the last 48 hours. You can change the timeframe to an hour, a week, a month, or a custom range of dates.\n\nThe check boxes allow you to filter the jobs by\n\n, and\n\nYou can check as many filters you want. Clicking one of the filters or changing the timeframe will update the page. Otherwise, the table does not auto-refresh. The two arrows in a circle to the right and below the timeframe will refresh the results in the table.\n\nThe results display basic information for the jobs such as the Narrative they are located within, App ID, Submission Time, Queue Time, Run Time, and Status. The name of the Narrative is hot-linked, and clicking on it will open a new tab with the Narrative. The App ID is hot-linked and clicking on it will open a new tab with the catalog page for the app.\n\nJob Status\n\nThe Job Status Cell has the status, an icon for the log, and a red cancel icon. Click the log icon to expand the table and show the contents of the log. Click the icon again to collapse the table. Canceling the job will prompt you to make sure you didn’t click it accidentally. Jobs that have been canceled may continue to show up as “Queued” or “Running” until they clear the system.\n\nJob Log\n\nThe Job ID and worker node are easily located and the log can be scrolled through within the pop-out. Job Logs can be downloaded in CSV, TSV, JSON, and TEXT formats using the download (downward-facing arrow into tray) icon.\n\nPreviousAnalyze Data Using KBase Apps\n\nNextRevise Your Narrative\n\nLast modified \n\n2yr ago
"""
human_message = HumanMessage(content=text)

In [63]:
res = llm([system_message, human_message])

In [64]:
res.content.split("\n")

['Answer: ',
 '    1. ### question: What is the Job Browser in KBase? ### answer: The Job Browser is one of the buttons on the left sidebar menu in KBase. It allows you to monitor and manage your jobs. By default, it shows all jobs submitted within the last 48 hours. You can change the timeframe to an hour, a week, a month, or a custom range of dates.',
 '    2. ### question: What information does the Job Browser display? ### answer: The Job Browser displays basic information for the jobs such as the Narrative they are located within, App ID, Submission Time, Queue Time, Run Time, and Status.',
 '    3. ### question: What is the Job Status Cell? ### answer: The Job Status Cell has the status, an icon for the log, and a red cancel icon. Click the log icon to expand the table and show the contents of the log. Click the icon again to collapse the table. Canceling the job will prompt you to make sure you didn’t click it accidentally. Jobs that have been canceled may continue to show up as 